In [1]:
import os
import collections
import random
import gym
import numpy as np

In [2]:
from cartPoleDqn import DQN

In [3]:
PROJECT_PATH = os.path.abspath("C:/Selbststudium/Udemy/Udemy_AI_")
MODELS_PATH = os.path.join(PROJECT_PATH, "models")
MODEL_PATH = os.path.join(MODELS_PATH, "dqn_cartpole.h5")

In [4]:
class Agent:
    def __init__(self, env: gym.Env):
        # DQN Env Variables
        self.env = env
        self.observations = self.env.observation_space.shape
        self.actions = self.env.action_space.n
        # DQN Agent Variables
        self.replay_buffer_size = 50_000
        self.train_start = 1_000 
            # Ab wie vielen gemachten Spielzügen im Replay-Buffer soll mit dem Training begonnen werden?
            # Das Netzwerk wird so nach 1_000 Spielzügen eingeschaltet
        self.memory = collections.deque(maxlen=self.replay_buffer_size)
            # Eine Liste an der an beiden Seiten etwas ändern kann
            # Ist der Speicher erstmal voll, werden die Daten von rechts nach links verschoben, 
            # bzw. die ältesten Daten werden zugunsten der neuen Daten gelöscht
        self.gamma = 0.95
        self.epsilon = 1.0
            # Wie viel Prozent der der Aktionen sollen schon zu Beginn zufällig gewählt sein?
            # Dieser Wert wird im Laufe des Trainings reduziert bis epsilon_min
        self.epsilon_min = 0.01
            # Minimaler Prozentsatz um zufällige Aktionen auszuführen
            # In einem Prozent der Fälle wollen wir noch eine zufällige Aktion haben
        self.epsilon_decay = 0.999 
            # Je näher an der 1, desto mehr Spielzüge benötigt man,
            # um mit epsilon an epsilon_min anzukommen
        # DQN Network Variables
        self.state_shape = self.observations
        self.learning_rate = 1e-3
        self.dqn = DQN(
            self.state_shape,
            self.actions,
            self.learning_rate
        )
        self.target_dqn = DQN(
            self.state_shape,
            self.actions,
            self.learning_rate
        )
        self.target_dqn.update_model(self.dqn)
        self.batch_size = 32

    def get_action(self, state):
        if np.random.rand() <= self.epsilon: 
            return np.random.randint(self.actions)
        else:
            return np.argmax(self.dqn(state)) # Die Aktion mit dem höchsten q-Value

    def train(self, num_episodes):
        last_rewards: Deque = collections.deque(maxlen=10)
        best_reward_mean = 0.0
        for episode in range(1, num_episodes + 1):
            total_reward = 0.0
            state = self.env.reset()
            state = np.reshape(state, newshape=(1, -1)).astype(np.float32) # Wieder für TF
            while True:
                action = self.get_action(state)
                next_state, reward, done, _ = self.env.step(action) # ausführen des steps
                next_state = np.reshape(next_state, newshape=(1, -1)).astype(np.float32)
                if done and total_reward < 500: # reward = 500 --> Gewonnen
                    reward = -100 # Verloren "böse bestrafen"
                self.remember(state, action, reward, next_state, done)
                self.replay()
                total_reward += reward
                state = next_state
                if done:
                    if total_reward < 500:
                        total_reward += 100
                    self.target_dqn.update_model(self.dqn)
                    print(f"Episode: {episode} --- Reward: {reward} --- Epsilon: {self.epsilon}")
                    current_reward_mean = np.mean(last_rewards)
                    if current_reward_mean > best_reward_mean:
                        best_reward_mean = current_reward_mean
                        self.dqn.save_model(MODEL_PATH)
                    break

    def remember(self, state, action, reward, next_state, done ):
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def replay(self):
        if len(self.memory) < self.train_start:
            return
        
        minibatch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, states_next, dones = zip(*minibatch)
        
        states = np.concatenate(states).astype(np.float32)
        states_next = np.concatenate(states_next).astype(np.float32)

        q_values = self.dqn(states)
        q_values_next = self.target_dqn(states_next)

        # Nun folgt die Umsetzung der theoretischen Formel:
        for i in range(self.batch_size):
            a = actions[i]
            done = dones[i]
            if done:
                q_values[i][a] = rewards[i]
            else: 
                q_values[i][a] = rewards[i] + self.gamma * np.max(q_values_next[i])

        self.dqn.fit(states, q_values) 
            # Training des Netzwerks auf den aktualisierten q_values, 
            # basierend auf den Aktionen, welche ausgeführt wurden

    def play(self, num_episodes, render=True):
        self.dqn.load_model(MODEL_PATH)
        for episode in range(1, num_episodes + 1):
            total_reward = 0.0
            state = self.env.reset()
            state = np.reshape(state, newshape=(1, -1)).astype(np.float32) # Wieder für TF
            while True:
                action = self.get_action(state)
                next_state, reward, done, _ = self.env.step(action) # ausführen des steps
                next_state = np.reshape(next_state, newshape=(1, -1)).astpye(np.float32)
                total_reward += reward
                state = next_state
                if done:
                    print(f"Episode: {episode} --- Reward: {reward} --- Epsilon: {self.epsilon}")
                    break

In [5]:
if __name__ == "__main__":
    env = gym.make("CartPole-v1")
    agent = Agent(env)
    agent.train(num_episodes=200)
    input("Play?")
    agent.play(num_episodes=10, render=True)

Episode: 1 --- Reward: -100 --- Epsilon: 0.9762739865836303
Episode: 2 --- Reward: -100 --- Epsilon: 0.9588496310845509
Episode: 3 --- Reward: -100 --- Epsilon: 0.9240034012385749
Episode: 4 --- Reward: -100 --- Epsilon: 0.9011784036598737
Episode: 5 --- Reward: -100 --- Epsilon: 0.8913148576343527
Episode: 6 --- Reward: -100 --- Epsilon: 0.862367254825433
Episode: 7 --- Reward: -100 --- Epsilon: 0.8064546837933355
Episode: 8 --- Reward: -100 --- Epsilon: 0.7936477332643059
Episode: 9 --- Reward: -100 --- Epsilon: 0.7818259902485653
Episode: 10 --- Reward: -100 --- Epsilon: 0.750403966288439
Episode: 11 --- Reward: -100 --- Epsilon: 0.7399663251239436
Episode: 12 --- Reward: -100 --- Epsilon: 0.7304042691684477
Episode: 13 --- Reward: -100 --- Epsilon: 0.7202448110511043
Episode: 14 --- Reward: -100 --- Epsilon: 0.7109376021267352
Episode: 15 --- Reward: -100 --- Epsilon: 0.6885367230816204
Episode: 16 --- Reward: -100 --- Epsilon: 0.674896028990821
Episode: 17 --- Reward: -100 --- Eps

C:\Users\gutsc\anaconda3\envs\tf_udemy\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\gutsc\anaconda3\envs\tf_udemy\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Episode: 29 --- Reward: -100 --- Epsilon: 0.5234447908547367
Episode: 30 --- Reward: -100 --- Epsilon: 0.5171978858215134
Episode: 31 --- Reward: -100 --- Epsilon: 0.5100039926842729
Episode: 32 --- Reward: -100 --- Epsilon: 0.5004006352830107
Episode: 33 --- Reward: -100 --- Epsilon: 0.4959150020176678
Episode: 34 --- Reward: -100 --- Epsilon: 0.4895066468838467
Episode: 35 --- Reward: -100 --- Epsilon: 0.4793291555710539
Episode: 36 --- Reward: -100 --- Epsilon: 0.47408281903197536
Episode: 37 --- Reward: -100 --- Epsilon: 0.4637617835123936
Episode: 38 --- Reward: -100 --- Epsilon: 0.452758565214779
Episode: 39 --- Reward: -100 --- Epsilon: 0.44735524511437874
Episode: 40 --- Reward: -100 --- Epsilon: 0.4429017700604677
Episode: 41 --- Reward: -100 --- Epsilon: 0.4354323744883354
Episode: 42 --- Reward: -100 --- Epsilon: 0.43066649544671043
Episode: 43 --- Reward: -100 --- Epsilon: 0.4204484963977607
Episode: 44 --- Reward: -100 --- Epsilon: 0.41626288125048977
Episode: 45 --- Rewar

OSError: Unable to open file (unable to open file: name = 'C:/Selbststudium/Udemy/Udemy_AI_\models\dqn_cartpole.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

KeyboardInterrupt: 